In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [4]:
!playwright install 

In [28]:
SEASONS=list(range(2014,2024))
SEASONS

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [29]:
DATA_DIR= 'data2'
STANDINGS_DIR= os.path.join(DATA_DIR, 'standings')
SCORES_DIR= os.path.join(DATA_DIR, 'score')

In [30]:
async def get_html(url, selector, sleep=5, retries=3):
    html=None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser= await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title)
                html= await page.inner_html(selector)
        except playwrightTimeout:
            print(f"Timout error on {url}")
            continue 
        else:
            break
    return html

In [31]:
async def get_html(url, selector, sleep=5, retries=5):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        print(f"Fetching {url}, Attempt {i}")
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url, timeout=60000)  # Increase the timeout to 60,000 ms (60 seconds)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:  # Correct the exception name
            print(f"Timeout error on {url}")
        else:
            break
    return html


In [32]:
async def scrape_season(season):
    url= f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    html= await get_html(url, '#content .filter')
    
    soup=BeautifulSoup(html)
    links=soup.find_all('a')
    href=[l['href']for l in links]
    standings_pages= [f'https://www.basketball-reference.com{l}'for l in href]
    
    for url in standings_pages:
        save_path= os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue
            
        html = await get_html(url,'#all_schedule')
        with open (save_path, 'w+') as f:
            f.write(html)

In [33]:
for season in SEASONS:
    await scrape_season(season)

Fetching https://www.basketball-reference.com/leagues/NBA_2014_games.html, Attempt 1
2013-14 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2015_games.html, Attempt 1
2014-15 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2016_games.html, Attempt 1
2015-16 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2017_games.html, Attempt 1
2016-17 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2018_games.html, Attempt 1
2017-18 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2019_games.html, Attempt 1
2018-19 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2020_games.html, Attempt 1
2019-20 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2021_games.

In [11]:
standing_files= os.listdir(STANDINGS_DIR)

In [34]:
async def scrape_game(standing_files):
    with open(standing_files, 'r') as f:
        html= f.read()

    soup=BeautifulSoup(html)
    links= soup.find_all('a')
    hrefs=[l.get('href')for l in links]
    box_scores= [l for l in hrefs if l and "boxscore" in l and '.html' in l]
    box_scores= [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path= os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue

        html= await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, 'w+')as f:
            f.write(html)

In [35]:
#chat gpt
async def scrape_game(standing_files):
    with open(standing_files, 'r') as f:
        html= f.read()

    soup=BeautifulSoup(html)
    links= soup.find_all('a')
    hrefs=[l.get('href')for l in links]
    box_scores= [l for l in hrefs if l and "boxscore" in l and '.html' in l]
    box_scores= [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        if os.path.exists(save_path):
            continue

        html = await get_html(url, '#content')
        if not html:
            continue 

        with open(save_path, 'w+') as f:
            f.write(html)

In [ ]:
standing_files= [s for s in standing_files if '.html' is s]

In [ ]:
for f in standing_files:
    filepath= os.path.join(STANDINGS_DIR, f)
    
    await scrape_game(filepath)

Fetching https://www.basketball-reference.com/boxscores/201403210TOR.html, Attempt 1
Thunder vs Raptors, March 21, 2014 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912110CHI.html, Attempt 1
Hawks vs Bulls, December 11, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912110MIN.html, Attempt 1
Jazz vs Timberwolves, December 11, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912110PHO.html, Attempt 1
Grizzlies vs Suns, December 11, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912110MIL.html, Attempt 1
Timeout error on https://www.basketball-reference.com/boxscores/201912110MIL.html
Fetching https://www.basketball-reference.com/boxscores/201912110MIL.html, Attempt 2
Pelicans vs Bucks, December 11, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912110SAC.html, Attempt 1
Thund

Heat vs 76ers, December 18, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912180WAS.html, Attempt 1
Bulls vs Wizards, December 18, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912180MIN.html, Attempt 1
Pelicans vs Timberwolves, December 18, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912180OKC.html, Attempt 1
Grizzlies vs Thunder, December 18, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912180DEN.html, Attempt 1
Magic vs Nuggets, December 18, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912180DAL.html, Attempt 1
Timeout error on https://www.basketball-reference.com/boxscores/201912180DAL.html
Fetching https://www.basketball-reference.com/boxscores/201912180DAL.html, Attempt 2
Celtics vs Mavericks, December 18, 2019 | Basketball-Reference.com
Fetching https://

Celtics vs Raptors, December 25, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912250PHI.html, Attempt 1
Bucks vs 76ers, December 25, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912250GSW.html, Attempt 1
Timeout error on https://www.basketball-reference.com/boxscores/201912250GSW.html
Fetching https://www.basketball-reference.com/boxscores/201912250GSW.html, Attempt 2
Timeout error on https://www.basketball-reference.com/boxscores/201912250GSW.html
Fetching https://www.basketball-reference.com/boxscores/201912250GSW.html, Attempt 3
Rockets vs Warriors, December 25, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912250LAL.html, Attempt 1
Clippers vs Lakers, December 25, 2019 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201912250DEN.html, Attempt 1
Pelicans vs Nuggets, December 25, 2019 | Basketball-Reference.com
Fet

Fetching https://www.basketball-reference.com/boxscores/201810160GSW.html, Attempt 1
Thunder vs Warriors, October 16, 2018 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201810170CHO.html, Attempt 1
Bucks vs Hornets, October 17, 2018 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201810170DET.html, Attempt 1
Nets vs Pistons, October 17, 2018 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201810170IND.html, Attempt 1
Grizzlies vs Pacers, October 17, 2018 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201810170ORL.html, Attempt 1
Heat vs Magic, October 17, 2018 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201810170NYK.html, Attempt 1
Hawks vs Knicks, October 17, 2018 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/201810170TOR.html, Attempt 1
Cavaliers vs Raptors, October 17

In [ ]:

#standing_files = os.listdir(STANDING_DIR)  # List of files in the STANDING_DIR directory

#for file_name in standing_files:
    #file_path = os.path.join(STANDING_DIR, file_name)

    #with open(file_path, 'r') as f:
        #html = f.read()

In [ ]:
async def scrape_games(standing_files):
    with open(standing_files, 'r') as f:
        html = f.read()
    soup=BeautifulSoup(html)
    links=soup.find_all('a')
    hrefs= [l.get('href')for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, '#content')
        if not html:
            continue

        with open(save_path, 'w+') as f:
            f.write(html)

In [ ]:
async def scrape_games(standing_files):
    with open(standing_files, 'r') as f:
        html = f.read()
    soup=BeautifulSoup(html)
    links=soup.find_all('a')
    hrefs= [l.get('href')for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue

In [ ]:
save_path = os.path.join(SCORES_DIR, url.split('/')[-1])


In [ ]:
import os

standing_files = os.listdir(STANDING_DIR)  # List of files in the STANDING_DIR directory

for file_name in standing_files:
    file_path = os.path.join(STANDING_DIR, file_name)

    # Call scrape_games for each file by passing the full file path (not in a list)
    await scrape_games(file_path)


In [ ]:
for season in SEASONS:
    for f in standing_files:
        filepath= os.path.join(STANDING_DIR, f)

        await scrape_games(filepath)

In [ ]:

standing_files = os.listdir(STANDING_DIR)  # List of files in the STANDING_DIR directory

for file_name in standing_files:
    file_path = os.path.join(STANDING_DIR, file_name)

    # Call scrape_games for each file by passing the full file path
    await scrape_games([file_path])


In [ ]:
box_scores